In [79]:
!pip install -r "../requirements.txt"

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.8 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.8 MB 540.5 kB/s eta 0:00:12
   --- ------------------------------------ 0.5/6.8 MB 540.5 kB/s eta 0:00:12
   ---- ----------------------------------- 0.8/6.8 MB 516.0 kB/s eta 0:00:12
   ---- ----------------------------------- 0.8/6.8 MB 516.0 kB/s eta 0:00:12
   ---- ----------------------------------- 0.8/6.8 MB 516.0 kB/s eta 0:00:12
   ------ --------------------------------- 1.0/6.8 MB 529.6 kB/s eta 0:00:11
   ------- -------------------------------- 1.3/6.8 MB 563.8 kB/s eta 0:00:10
   ------- --------------------

In [91]:
import os

documents_dir = "documents/"
target_folders = ["Condotril", "Duobiotic", "Neurofil"]  # nomes dos produtos
documents = {folder: "" for folder in target_folders}  # Inicializa um dicionário para cada produto

for folder in target_folders:
    folder_path = os.path.join(documents_dir, folder)
    
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        for doc_name in os.listdir(folder_path):
            if doc_name.endswith(".txt"):
                print(f"txt encontrado ({doc_name})!")
                file_path = os.path.join(folder_path, doc_name)
                with open(file_path, "r", encoding="utf-8") as doc:
                    content = doc.read()
                    documents[folder] += content + "\n"

txt encontrado (Condotril_bula.txt)!
txt encontrado (Duobiotic_bula.txt)!
txt encontrado (Neurofil_bula.txt)!


In [92]:
documents

{'Condotril': '=D MYPharmã\n\nCONDOTRIL\n\nO QUE É CONDOTRIL?\n\nCONDOTRIL é um suplemento alimentar que contém Sulfato de Glucosamina, Sulfato de Condroitina,\nMSM e outros compostos como a Vitamina C e Manganês. A Vitamina C contribui para a normal for-\nmação de colagénio para o funcionamento normal dos ossos e das cartilagens. O Manganês con-\ntribui para a manutenção de ossos normais e a normal formação de tecidos conjuntivos.\n\nOs suplementos alimentares são géneros alimentícios que se destinam a complementar e/ou a\nsuplementar o regime alimentar normal não devendo ser utilizados como substitutos de um regime\nalimentar variado. Constituem fontes concentradas de determinadas substâncias nutrientes ou\noutras, com efeito nutricional ou fisiológico. É importante um regime alimentar equilibrado e um\nmodo de vida saudável.\n\nIngredientes: Agente de volume: Maltodextrina; Sulfato de glucosamina (marinho); Sulfato\nde condroitina; Agente de volume: Fosfato dicálcico; Cartilagem de 

# Question Generator

In [94]:
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="llama3.1:8b")

# original questions

questions = ["Quais sao os ingredientes do Condotril?",
"Quais sao os ingredientes do Duobiotic?",
"Quais sao os ingredientes do Neurofil?",
"Quais sao os efeitos do Condotril?",
"Quais sao os efeitos do Duobiotic?",
"Quais sao os efeitos do Neurofil?",
"Qual e o processo de toma do Condotril?",
"Qual e o processo de toma do Duobiotic?",
"Qual e o processo de toma do Neurofil?",
"Quais sao as condicoes de armazenamento do Condotril?",
"Quais sao as condicoes de armazenamento do Duobiotic?",
"Quais sao as condicoes de armazenamento do Neurofil?",
"Em que casos nao se pode tomar Condotril?",
"Em que casos nao se pode tomar Duobiotic?",
"Em que casos nao se pode tomar Neurofil?",
"Quantas doses tem a embalagem do Condotril?",
"Quantas doses tem a embalagem do Duobiotic?",
"Quantas doses tem a embalagem do Neurofil?"]

### Variants

In [95]:
def generate_variants(model, questions, n=4):
    variants = {}
    for question in questions:
        system_prompt =  """Your role is to rephrase this question in different ways, keeping the meaning: {question}.
        Never give a question in english, only in portuguese.""".format(question=question)

        response = model.generate(prompts=[system_prompt], num_return_sequences=n)
        variants[question] = [variant for variant in response.generations]
    return variants

In [96]:
#!ollama pull llama3.1

In [97]:
question_variants = generate_variants(model, questions, n=4)

In [98]:
for original, variants in question_variants.items():
    print(f"Original: {original}")
    for i, variant in enumerate(variants, 1):
        print(f"Variant {i}: {variant}")
    print()

Original: Quais sao os ingredientes do Condotril?
Variant 1: [GenerationChunk(text='Aqui estão algumas reformulações da pergunta:\n\n* Quais são as componentes fundamentais do Condotril?\n* O que compõe o Condotril?\n* Qual é a composição do Condotril?\n* Em que se baseia a fórmula do Condotril?\n* Quais são os principais ingredientes utilizados no fabrico do Condotril?', generation_info={'model': 'llama3.1:8b', 'created_at': '2024-11-25T16:39:11.7787811Z', 'response': '', 'done': True, 'done_reason': 'stop', 'context': [128006, 882, 128007, 271, 7927, 3560, 374, 311, 312, 28810, 420, 3488, 304, 2204, 5627, 11, 10494, 279, 7438, 25, 3489, 2852, 104372, 2709, 25795, 288, 656, 1221, 16510, 31660, 30, 627, 286, 15037, 3041, 264, 3488, 304, 30063, 11, 1193, 304, 2700, 773, 36554, 13, 128009, 128006, 78191, 128007, 271, 32, 47391, 57554, 93112, 15180, 5724, 15607, 3067, 824, 61010, 1473, 9, 3489, 2852, 30710, 439, 3777, 288, 12762, 2852, 656, 1221, 16510, 31660, 5380, 9, 507, 1744, 1391, 11

### Display and dictionary + Normalize variants

In [ ]:
import re
import unicodedata

def remove_acentos(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

def remove_pontuation(text):
        return re.sub(r"[^\w\s]", "", text) # remove todos os caracteres exceto letras e numeros

var = {}
var_normalized = {}

for original, variants in question_variants.items():
    QV = []
    #return re.sub(r"[^\w\s\!\?\.\']", "", text)
    for variant in variants:
        responses = re.split(r'[\n\*|\n,]', variant[0].text)
        responses = [resp.strip() 
                     for resp in responses[1:] if resp.strip()]
        QV.extend(responses)
    var[original] = QV

for original, variants in var.items():
    QV = []
  
    original = remove_acentos(remove_pontuation(original))
    
    for variant in variants:
        variant = remove_pontuation(variant) 
        variant = remove_acentos(variant)
        
        QV.append(variant)
    
    var_normalized[original] = QV

    print(f"Original: {original}")
    print("Variantes:")
    for variant in var_normalized[original]:
        print("- ", variant)
    print()

Original: Quais sao os ingredientes do Condotril
Variantes:
-  Quais sao as componentes fundamentais do Condotril
-  O que compoe o Condotril
-  Qual e a composicao do Condotril
-  Em que se baseia a formula do Condotril
-  Quais sao os principais ingredientes utilizados no fabrico do Condotril

Original: Quais sao os ingredientes do Duobiotic
Variantes:
-  Qual e a composicao quimica do Duobiotic
-  O que contem o Duobiotic como ingrediente ativo
-  Sao quais os componentes do Duobiotic
-  Quais sao as substancias presentes no Duobiotic
-  Qual e a formula do Duobiotic

Original: Quais sao os ingredientes do Neurofil
Variantes:
-  Qual e a composicao do Neurofil
-  Quais sao as substancias que compoem o Neurofil
-  O que esta presente no composto conhecido como Neurofil
-  Que ingredientes ou componentes fazem parte da estrutura do Neurofil
-  Quais os constituintes quimicos do Neurofil

Original: Quais sao os efeitos do Condotril
Variantes:
-  O que acontece com as pessoas que utiliz

## Variants dataset

In [140]:
import pandas as pd

variants_df = pd.DataFrame(list(var_normalized.items()), columns=["original", "variants"])
max_variants = max(variants_df["variants"].apply(len))
variants_expanded = pd.DataFrame(variants_df["variants"].tolist(), columns=[f"variant_{i+1}" for i in range(max_variants)])
variants_df = pd.concat([variants_df["original"], variants_expanded], axis=1)

In [141]:
variants_df

,original,variant_1,variant_2,variant_3,variant_4,variant_5,variant_6,variant_7,variant_8,variant_9,variant_10
0,Quais sao os ingredientes do Condotril,Quais sao as componentes fundamentais do Condo...,O que compoe o Condotril,Qual e a composicao do Condotril,Em que se baseia a formula do Condotril,Quais sao os principais ingredientes utilizado...,None,None,None,None,None
1,Quais sao os ingredientes do Duobiotic,Qual e a composicao quimica do Duobiotic,O que contem o Duobiotic como ingrediente ativo,Sao quais os componentes do Duobiotic,Quais sao as substancias presentes no Duobiotic,Qual e a formula do Duobiotic,None,None,None,None,None
2,Quais sao os ingredientes do Neurofil,Qual e a composicao do Neurofil,Quais sao as substancias que compoem o Neurofil,O que esta presente no composto conhecido como...,Que ingredientes ou componentes fazem parte da...,Quais os constituintes quimicos do Neurofil,None,None,None,None,None
3,Quais sao os efeitos do Condotril,O que acontece com as pessoas que utilizam o C...,Quais sao as consequencias de usar o Condotril,Como afeta o corpo humano o uso do Condotril,Quais sao os danos causados pelo Condotril,O que acontece com a saude quando se utiliza o...,None,None,None,None,None
4,Quais sao os efeitos do Duobiotic,Qual e o resultado de usar o Duobiotic,O que acontece quando se toma o Duobiotic,Quais sao as consequencias de consumir o Duobi...,Como afeta o organismo o uso do Duobiotic,Quais sao os efeitos colaterais ou secundarios...,Qual e a resposta do corpo ao Duobiotic,None,None,None,None
5,Quais sao os efeitos do Neurofil,1 Qual e o impacto do Neurofil no organismo,2 Como afeta o corpo humano o consumo de Neurofil,3 Quais sao as consequencias da ingestao de Ne...,4 O que acontece com o corpo quando se toma um...,5 Quais sao os efeitos secundarios do uso de p...,None,None,None,None,None
6,Qual e o processo de toma do Condotril,Como se desenvolveu o processo de construcao d...,Quais foram as etapas que levaram a criacao do...,O que aconteceu para que o Condotril fosse con...,Descreva como foi feito o processo de elaborac...,Quais sao os principais passos que compuseram ...,Como foi organizado e executado o projeto do C...,Qual foi o fluxo de trabalho usado para criar ...,None,None,None
7,Qual e o processo de toma do Duobiotic,Como funciona a tomada do Duobiotic,O que e necessario para tomar Duobiotic,Quais sao os passos para administrar Duobiotic,Qual e o procedimento para ingestao de Duobiotic,Como se toma corretamente o Duobiotic,None,None,None,None,None
8,Qual e o processo de toma do Neurofil,Como se adquire o Neurofil,O que e necessario para obter o Neurofil,Quais sao os passos para adquirir o Neurofil,Qual e a forma correta de obter o Neurofil,Quem pode tomar ou comprar um Neurofil,None,None,None,None,None
9,Quais sao as condicoes de armazenamento do Con...,1 Como devem ser armazenadas as pecas da Condo...,2 Quais sao os requisitos para o armazenamento...,3 O que e necessario saber sobre a conservacao...,4 Em que condicoes deve ser guardado o Condotril,5 Qual e a recomendacao para o armazenamento d...,6 Quais sao as normas para o armazenamento do ...,7 Como devemos proceder para armazenar o Condo...,8 Quais sao os criterios para o armazenamento ...,Essas perguntas mantem a mesma essencia da ori...,apenas com diferentes frases e estruturas


In [142]:
variants_df.to_csv('variants.csv')

### Handle speling

In [ ]:
from fuzzywuzzy import fuzz
from autocorrect import Speller
from spellchecker import SpellChecker
import re
import csv

def load_variants_from_csv(file_path):
    quest_variants = {}
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            original = row['original']  # A coluna principal com a pergunta original
            # Filtrar todas as variantes não vazias
            variants = [row[key] for key in row if key.startswith('variant_') and row[key].strip()]
            quest_variants[original] = variants  # Mapear original -> lista de variantes
    return quest_variants

# spell = Speller(lang='pt')
spell = SpellChecker(language='pt')

def normalize_text(text):
        return re.sub(r"[^\w\s']", "", text)

# def normalize_question(user_question, quest_variants):
#     for original, variants in quest_variants.items():
#         if user_question.lower() == original or user_question.lower() in [variant.lower() for variant in variants]:
#             return original
#     return user_question


def choose_best_question_variant(user_question, quest_variants):
    best_score = 0
    best_match = None

    print("original:", user_question)

    # user_question = spell(user_question) # CATARINA
    # user_question = spell.correction(user_question) # MARTA
    # print("spell.correction: ", user_question) # NAO FUNCIONA ###############

    user_question = normalize_text(user_question)
    print("normalize_text: ", user_question)

    user_question = remove_acentos(remove_pontuation(user_question))
    print("remove_acentos: ", user_question, "\n")
    
    user_question = user_question.lower()

    # Comparação com perguntas originais e variantes
    for original, variants in quest_variants.items():
        # Verifica a similaridade com a pergunta original
        score_with_original = fuzz.ratio(user_question, original.lower())
        if score_with_original > best_score:
            best_score = score_with_original
            best_match = original
            # Se a original for altamente semelhante, priorize-a
            if best_score >= 98:
                break

        # Verifica a similaridade com cada variante
        for variant in variants:
            score_with_variant = fuzz.ratio(user_question, variant.lower())
            if score_with_variant > best_score:
                best_score = score_with_variant
                best_match = original

    return best_match


In [373]:
teste_com_erros = "quantós comprmídos d~evo tomâr de duobitiic?"
file_path = 'variants.csv'
variants = load_variants_from_csv(file_path)
result = choose_best_question_variant(teste_com_erros, variants)
print("Melhor variante:", result)

original: quantós comprmídos d~evo tomâr de duobitiic?
normalize_text:  quantós comprmídos devo tomâr de duobitiic
remove_acentos:  quantos comprmidos devo tomar de duobitiic 

Melhor variante: Qual e o processo de toma do Duobiotic


# Create Chunks

In [374]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1200, # o chunk_size tem que ser grande para termos o maximo de informaçao sobre um produto num vetor apenas
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

docs = []
for name, content in documents.items():
    # Adiciona o nome do medicamento no início do conteúdo antes de dividir
    full_content = f"Medicamento: {name}\n\n{content}"
    chunks = text_splitter.create_documents([full_content])
    
    # Adiciona o nome do medicamento diretamente ao conteúdo de cada chunk
    for chunk in chunks:
        chunk.metadata = {"medicamento": name}
        chunk.page_content = f"{name}\n\n{chunk.page_content}"
    
    docs.extend(chunks)

for _, (product, content) in enumerate(documents.items()):
    num_characters = len(content)
    print(f"Número de caracteres no documento '{product}': {num_characters}")

print("Número de documents: ", len(docs))

Número de caracteres no documento 'Condotril': 3161
Número de caracteres no documento 'Duobiotic': 3207
Número de caracteres no documento 'Neurofil': 2871
Número de documents:  10


In [375]:
docs

[Document(metadata={'medicamento': 'Condotril'}, page_content='Condotril\n\nMedicamento: Condotril\n\n=D MYPharmã\n\nCONDOTRIL\n\nO QUE É CONDOTRIL?\n\nCONDOTRIL é um suplemento alimentar que contém Sulfato de Glucosamina, Sulfato de Condroitina,\nMSM e outros compostos como a Vitamina C e Manganês. A Vitamina C contribui para a normal for-\nmação de colagénio para o funcionamento normal dos ossos e das cartilagens. O Manganês con-\ntribui para a manutenção de ossos normais e a normal formação de tecidos conjuntivos.\n\nOs suplementos alimentares são géneros alimentícios que se destinam a complementar e/ou a\nsuplementar o regime alimentar normal não devendo ser utilizados como substitutos de um regime\nalimentar variado. Constituem fontes concentradas de determinadas substâncias nutrientes ou\noutras, com efeito nutricional ou fisiológico. É importante um regime alimentar equilibrado e um\nmodo de vida saudável.'),
 Document(metadata={'medicamento': 'Condotril'}, page_content='Condotr

# Create VectorStore/Retriever

In [378]:
# ollama needs to be installed
# https://ollama.com/
# https://github.com/ollama/ollama/tree/main/docs
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = OllamaEmbeddings(
    # model="nomic-embed-text",
    model="llama3.1:8b",
)

vectorstore = FAISS.from_documents(docs, embedding=embedding_model)
vectorstore.save_local(folder_path = 'vectorstore') # save the vectorstore locally

In [379]:
retriever = FAISS.load_local(
    folder_path='vectorstore',
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
    ).as_retriever(search_kwargs={"k" : 5})

# Load Document

In [382]:
question = "Quais são os ingredientes do Condotril"
question = choose_best_question_variant(question, variants)
retriever.invoke(question)

original: Quais são os ingredientes do Condotril
normalize_text:  Quais são os ingredientes do Condotril
remove_acentos:  Quais sao os ingredientes do Condotril 



[Document(metadata={'medicamento': 'Condotril'}, page_content='Condotril\n\nConsumir de preferência antes do fim de...: Ver embalagem. -\nLote: Ver embalagem.\n\nRepresentado e distribuído por:\nMYPHARMA S.A.\n\nTv. António Ferreira Rito, 43\n4705-112 Braga - PORTUGAL\n\ngeralomypharma,.pt Good Mm..nSI..MZ &\n\nwww.mypharma.pt'),
 Document(metadata={'medicamento': 'Neurofil'}, page_content='Neurofil\n\nMedicamento: Neurofil\n\nNEUROFIL\n\n60 CÁPSULAS MOLES\n\nO QUE É NEUROFIL?\n\nNeurofil é um suplemento alimentar que contém óleo de peixe concentrado em ácidos gordos Ómega-a. com quantidades\nrelevantes de DHA / EPA num rácio de 3:1. Inclui ainda magnésio, vitamina B12 e ácido fólico. Os suplementos alimentares são\ngéneros alimentícios que se destinam a complementar e/ou a suplementar o regime alimentar normal não devendo ser\nutilizados como substitutos de um regime alimentar variado. Constituem fontes concentradas de determinadas substâncias\nnutrientes ou outras, com efeito nutrici

In [24]:
import re
from langchain.tools import BaseTool

class ArticleInformation(BaseTool):
    name: str = "ArticleInformation"
    description: str = "This tool is used to answer questions about MyPharma food suplements. The input is the original user question"
    
    # def _run(self, question):
    #     retrieved_documents = retriever.invoke(question)
    #     return [document.page_content for document in retrieved_documents]
    
    def _run(self, question):
        retrieved_documents = retriever.invoke(question)  # Obtém documentos
        medicamentos = self._extract_medicamento(question)  # Extrai os nomes dos medicamentos (lista)
        
        # Filtra os documentos com base na metadata
        filtered_documents = [
            document for document in retrieved_documents
            if document.metadata.get('medicamento', '').lower() in {med.lower() for med in medicamentos}
        ]
        
        # Retorna os conteúdos dos documentos filtrados
        return [document.page_content for document in filtered_documents]

    
    def _extract_medicamento(self, question: str) -> list[str]:
        """Tenta identificar os medicamentos mencionados na pergunta."""
        # Lista fixa de medicamentos disponíveis
        medicamentos_disponiveis = {"Condotril", "Neurofil", "Duobiotic"}

        # Procura por todos os medicamentos mencionados na pergunta
        matches = re.findall(r'(Condotril|Neurofil|Duobiotic)', question, flags=re.IGNORECASE)
        
        # Filtra para capturar somente os medicamentos válidos, eliminando duplicatas
        medicamentos_identificados = {match.capitalize() for match in matches if match.capitalize() in medicamentos_disponiveis}
        
        if medicamentos_identificados:
            return list(medicamentos_identificados)
        else:
            raise ValueError("Não foi possível identificar nenhum medicamento na pergunta.")


# Develop Chatbot workflow (langgraph)

Tools

In [25]:
tools = [ArticleInformation()]
tools_by_name = {tool.name: tool for tool in tools}

AgentState

In [26]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

System Prompt

In [27]:
from langchain_core.messages import SystemMessage

system_prompt =  SystemMessage("""Your role is to act as a thorough research assistant, providing complete and detailed answers based on the given context. 
  
Your final answer should be as complete as possible. Do not oversimplify or summarize unnecessaraly, considering the results of the tools you used.
Do not oversimplify or summarize the answer.

If you are unsure about what tool to use or how to correct an error, you should ask the user for help.""")

Nodes

In [ ]:
from typing import Union, Literal
from langchain_ollama import ChatOllama # you can use other chat models from langchain. they need, however, to have function call capabilities (https://python.langchain.com/docs/integrations/chat/)
from langchain_core.messages import ToolMessage, HumanMessage
import json


# ollama needs to be installed
# https://ollama.com/
# https://github.com/ollama/ollama/tree/main/docs
def call_llm(state: AgentState) -> dict[str, Union[list, bool]]:
        """
        Calls the LLM model.
        
        Parameters
        ----------
        state : AgentState
            The state of the agent.
            
        Returns
        -------
        dict[str, Union[list, bool]]
            The updated state after calling the LLM model.
        """
        llm = ChatOllama( model="llama3.1:8b", temperature=0) #instantiates the LLM model
        llm = llm.bind_tools([ArticleInformation()]) #binds the tools to the LLM model
        llm_response = llm.invoke(state['messages']) #calls the LLM model
        
        return {'messages': [llm_response]} #returns the updated state
    
def tool_node(state: AgentState) -> dict[str, Union[list, bool]]:
    '''
    Runs the tool requested in the last AI message (with the tool call).
    
    Parameters
    ----------
    state : AgentState
        The state of the agent.
        
    Returns
    -------
    dict[str, Union[list, bool]]
        The updated state after running the tool.
    '''
    if messages := state.get('messages', []):
        message = messages[-1]
    else:
        raise ValueError('No messages found in input state.')
    
    outputs = []
    
    # tool_call example
    # [{'name': 'ThesisInformation',
    # 'args': {'query': 'What is the...'},
    # 'id': 'call_UwQzgi1QD9uWfBP1r6CWO5ep',
    # 'type': 'tool_call'}]
    
    for tool_call in message.tool_calls:
        tool_result = tools_by_name[tool_call['name']].invoke(
            tool_call['args']
        )
        
        outputs.append(
            ToolMessage(
                content = json.dumps(tool_result),
                name = tool_call['name'],
                tool_call_id = tool_call['id']
            )
        )

    return {'messages': outputs}


def route_tools(state: AgentState) -> Literal['tools', '__end__']:
        '''
        To be used in the conditional_edge method when compiling the graph.
        Routes to the tool node if the last message contains a tool call.
        Otherwise, routes to the end.
        
        Parameters
        ----------
        state : ReactState
            The state of the agent.
            
        Returns
        -------
        Literal['tools', '__end__']
            The next node to route to.
        '''        
        if isinstance(state, list):
            ai_message = state[-1]
        elif messages := state.get('messages', []):
            ai_message = messages[-1]
        else:
            raise ValueError(f'No messages found in input state to tool edge: {state}')
        
        if hasattr(ai_message, 'tool_calls') and len(ai_message.tool_calls) > 0:
            return 'tools'
        return '__end__'

Compile Graph

In [29]:
from langgraph.graph import StateGraph, START

graph = StateGraph(AgentState)
graph.add_node('llm', call_llm)
graph.add_node('tools', tool_node)
graph.add_edge('tools', 'llm')
graph.add_edge(START, 'llm')
graph.add_conditional_edges('llm', route_tools, {'tools': 'tools', '__end__': '__end__'})

#compile graph
graph = graph.compile()

In [30]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))


# Answers

In [31]:
# Qual é a composição do suplemento alimentar X?
# Quais são os efeitos do suplemento alimentar X? / O que faz o suplemento alimentar X?
# O que acontece se consumir X e tomar o suplemento alimentar Y?
# Como é o processo de toma do suplemento alimentar X?

# Ingredientes
question1 = "Quais são os ingredientes do Condotril?"
question2 = "Quais são os ingredientes do Duobiotic?"
question3 = "Quais são os ingredientes do Neurofil?"

# Benefícios
question4 = "Quais são os benefícios do Condotril?"
question5 = "Quais são os benefícios do Duobiotic?"
question6 = "Quais são os benefícios do Neurofil?"

# Efeitos
question7 = "Quais são os efeitos do Condotril?"
question8 = "Quais são os efeitos do Duobiotic?"
question9 = "Quais são os efeitos do Nueurofil?"

# Processo de toma
question10 = "Qual é o processo de toma do Condotril?"
question11 = "Qual é o processo de toma do Duobiotic?"
question12 = "Qual é o processo de toma do Neurofil?"

# Condições de armazenamento
question13 = "Quais são as condições de armazenamento do Condotril?"
question14 = "Quais são as condições de armazenamento do Duobiotic?"
question15 = "Quais são as condições de armazenamento do Neurofil?"

# Precaussões
question16 = "Em que casos não se pode tomar Condotril?"
question17 = "Em que casos não se pode tomar Duobiotic?"
question18 = "Em que casos não se pode tomar Neurofil?"

In [33]:
response1 = graph.invoke({'messages': [system_prompt, HumanMessage(content=question1)]}, debug=True)

In [35]:
print("Pergunta: ", question1)
print("Resposta: ", response1['messages'][-1].content)

In [36]:
response2 = graph.invoke({'messages': [system_prompt, HumanMessage(content=question2)]}, debug=True)

In [37]:
print("Pergunta: ", question2)
print("Resposta: ", response2['messages'][-1].content)

In [38]:
response3 = graph.invoke({'messages': [system_prompt, HumanMessage(content=question3)]}, debug=True)

In [39]:
print("Pergunta: ", question3)
print("Resposta: ", response3['messages'][-1].content)

In [41]:
bonus_question = "Qual é a diferença entre Duobiotic e Neurofil?"
response4 = graph.invoke({'messages': [system_prompt, HumanMessage(content=bonus_question)]}, debug=True)

In [ ]:
import textwrap
line_width = 175  # ajuste para a largura desejada
response4 = textwrap.fill(response4['messages'][-1].content, width=line_width)

print("Pergunta:", bonus_question)
print("Resposta:\n", response4)